Antes de ver este ejemplo, deberias haber visto los Notebooks [Ejemplo - Onda de Sonido](Ejemplo%20-%20Onda%20de%20Sonido.ipynb) y [Ejemplo - Modelo deposito agua](Ejemplo%20-%20Modelo%20deposito%20agua.ipynb)

# Modelo hidrológico
En este Notebook vamos a crear un modelo hidrologico y vamos a utilizar widgets interactivos para intentar entender mejor como funciona el modelo, como influencian los parametros del modelo en el resultado y como podemos calibrar el modelo. 

Primero de todo, ¿que es un modelo hidrologico? Imagina que queremos predecir el hidrograma de un rio que recoje el agua de una cuenca cuando llueve. Para este proposito utilizamos un modelo hidrologico, que no es mas que un conjunto de ecuaciones que describen de una manera simplificada los procesos hidrologicos que ocurren en la cuenca. Estas ecuaciones incluyen diferentes parametros que describen algunas de las propiedades de la cuenca, por ejemplo las caracterisiticas del suelo.

![diagrama cuenca](util/diagrama_cuenca.gif)

En este ejemplo, vamos a utilizar un modelo simple (una adaptacion del modelo [HyMOD](https://doi.org/10.1002/9781118665671.ch14)) que tiene 5 parametros:

- **Soil storage capacity** (mm): capacidad que tiene el suelo de retener agua de lluvia
- **Evaporation ratio**: ratio de evaporacion o proporcion de lluvia que se evapora
- **Infiltration ratio**: ratio de infiltracion proporcion del agua de lluvia efectiva (que no se evapora) que se infiltra en el suelo.
- **Travel time - surface flow** (days): tiempo de concentracion del agua superficial o tiempo que tarda el agua superficial en llegar a la desembocadura de la cuenca.
- **Travel time - underground flow** (days): tiempo de concentracion del agua subterranea o tiempo que tarda el agua subterranea en llegar a la desembocadura de la cuenca.

En la imagen de abajo podemos ver como el modelo representa los procesos hidrologicos para asi obtener finalmente la prediccion del hidrograma del rio para los proximos meses. Como puedes ver el suelo es presentado como un deposito con una cierta capacidad que si es sobrepasada desborda generando flujo superficial y que tambien desaloja agua por su parte inferior generando agua subterranea.

![diagrama modelo hidrologico](util/diagrama_modelo_hidrologico.gif)

Vamos a crear el modelo y probarlo para comprender mejor como funciona.

Lo primero como siempre es importar las librerias de funciones que vamos a necesitar

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from ipywidgets import interact

### Periodo de simulacion
Vamos a definir el periodo de simulacion, en este caso 120 dias (`T`), desde el 1 de Marzo.

In [ ]:
T = 120 # dias del periodo de simulacion
dates = pd.date_range(start = '2022-03-01', periods = T)
dates

### Datos de entrada (inputs)
Los inputs de nuestro modelo son los datos de lluvia (`prec`) y evapotranspiracion potencial (`etp`). Deben ser series temporales (diarias) y en formato array. Como solo vamos a hacer una prueba del modelo vamos a generar datos aleatorios con una distribucion uniforme pero podriamos haber usado datos reales, importandolos por ejemplo de un archivo Excel.

In [ ]:
prec = np.random.uniform(low = 0, high = 30, size = [T,1])
etp = np.random.uniform(low = 0, high = 5, size = [T,1])

### Parametros del modelo
Definimos los 5 paramtetros del modelo y le damos un valor inicial.

![parametros_modelo_hidrologico](util/parametros_modelo_hidrologico.png)

In [ ]:
sm_max = 10  # Maxima capacidad de retencion del del suelo (mm) [10 - 90]
gamma  = 0.5 # Ratio de evaporacion () [0 - 1]
alpha  = 0.5 # Ratio de infiltracion () [0 - 1]
t_sup  = 1   # Tiempo concentracion del flujo superficial [0.8 - 2]
t_sub  = 5   # Tiempo concentracion del flujo subterraneo [2 - 10]

### Declaracion de las variables del modelo
Ahora declaramos (o inicalizamos) las variables del modelo. No hace falta que presteis mucha atencion a estas variables, simplemente quedaos con la idea de que van a formar parte de las ecuaciones del modelo que representan de manera simplificada los procesos hidrologicos de la cuenca. Como en cada paso (time-step) de la simulacion vamos a actualizar sus valores, necesitamos inicializarlas como un array de `T` ceros

In [ ]:
inf   = np.zeros((T,1))   # Tasa de infiltracion [mm/t]
et    = np.zeros((T,1))   # Tasa de evapotranspiration [mm/t]
sm    = np.zeros((T+1,1)) # Contenido de humdedad en el suelo [mm] (suponemos que el suelo esta seco inicialmente)
sL    = np.zeros((T+1,1)) # Slow reservoir moisture [mm]
sF    = np.zeros((T+1,1)) # Fast reservoir moisture [mm]
Q_sub = np.zeros((T,1))   # Flujo subterraneo [mm/t]
Q_sup = np.zeros((T,1))   # Flujo superficial [mm/t]
Q_tot = np.zeros((T,1))   # Flujo total [mm/t]

## Simulacion
Para la simulacion vamos a escribir las ecuaciones del modelo dentro de un bucle (loop) `for` y asi realizar los calculos automaticamente para todos los dias (`T` = 120) desde el 1 de Marzo. Las ecuaciones son sencillas y son en su mayoria muy parecidas a la ecuacion de simulacion de un deposito (que vimos en el Notebook "Ejemplo - Modelo deposito de agua"). No tienes que entender las ecuaciones, de nuevo solo tener en mente que representan los procesos hidrologicos de la cuenca de una manera simplificada.

In [ ]:
for t in range(T):

    ##### Humedad del suelo #####          
    # Actualiza la humedad del suelo sumando la precipitación y restando la infiltración,
    # asegurando que se mantenga dentro del rango [0, sm_max].
    sm_temp = max(min(sm[t] + prec[t] - inf[t], sm_max), 0)

    ##### Infiltración #####
    # Calcula la infiltración teniendo en cuenta la humedad disponible y la capacidad del suelo.
    # El primer término considera el exceso de agua más allá de la capacidad del suelo.
    # El segundo término evita valores negativos.
    inf[t] = inf[t] + max(sm[t] + prec[t] - inf[t] - sm_max, 0) + min(sm[t] + prec[t] - inf[t], 0)

    ##### Evapotranspiración #####
    # Calcula el factor de corrección de la evapotranspiración (W),
    # asegurando que no supere el valor de 1. Depende de la humedad relativa del suelo y de gamma.
    W = min(np.abs(sm[t]/sm_max) * gamma, 1)
    
    # Calcula la evapotranspiración real como el producto de W y la evapotranspiración potencial.
    et[t] = W * etp[t]

    ##### Humedad del suelo (t+1) #####
    # Actualiza la humedad del suelo después de la evapotranspiración,
    # asegurando que se mantenga dentro de los límites establecidos.
    sm[t+1] = max(min(sm_temp - et[t], sm_max), 0)

    ##### Flujo subterráneo #####
    # Calcula el flujo subterráneo (Q_sub) en función del almacenamiento lateral (sL)
    # y la escala de tiempo del flujo subterráneo (t_sub).
    Q_sub[t] = 1/t_sub * sL[t]

    # Actualiza el almacenamiento lateral (sL) sumando una fracción (1-alpha) de la infiltración
    # y restando el flujo subterráneo.
    sL[t+1] = sL[t] + (1 - alpha) * inf[t] - Q_sub[t]

    ##### Flujo superficial #####
    # Actualiza el almacenamiento superficial (sF) sumando una fracción (alpha) de la infiltración
    # y restando la fracción del flujo superficial en función de la escala de tiempo (t_sup).
    sF[t+1] = sF[t] + alpha * inf[t] - 1/t_sup * sF[t]

    # Calcula el flujo superficial (Q_sup) en función del almacenamiento superficial y la escala de tiempo (t_sup).
    Q_sup[t] = 1/t_sup * sF[t]

    ##### Flujo total #####
    # Calcula el flujo total como la suma del flujo superficial y el flujo subterráneo.
    Q_tot[t] = Q_sup[t] + Q_sub[t]

### Visualizacion de los resultados
Ahora podemos ver el resultado de la simulacion, en este caso representamos el hidrograma del flujo superficial, el flujo subterraneao y el hidrograma total (suma de los dos)

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(dates,Q_tot, label = 'total', color = 'blue')
plt.plot(dates,Q_sup, linestyle = '--', color = 'orange',   label = 'superficial')
plt.plot(dates,Q_sub, linestyle = ':',  color = 'green', label = 'subterraneo')
plt.ylim(0,25) # para fijar los limites del eje y, en este caso 0 y 25
plt.legend()
plt.show()

Si ahora quisieramos repetir la simulacion pero con distintos valores de los parametros para comprobar como cambia el resultado necesitariamos volver a la celda "Parametros del modelo" cambiar manualmente los valores y volver a ejecutar las celdas posteriores hasta generar la grafica del hidrograma... No suena muy eficiente. Vamos a hacer algo diferente.

### Modelo hidrologico interactivo
Para poder entender la influencia de los parametros del modelo en el resultado vamos a hacer nuestro modelo mas interactivo usando widgets, y mas concretamente deslizadores (sliders). Ahora vamos a crear una funcion `modelo_hidrologico` que va a contener nuestro modelo hidrologico y por medio de la funcion `interact` de la libreria Ipwidgets vamos a poder interactuar con los parametros del modelo mas facilmente.

In [ ]:
@interact(sm_max = (10, 90, 1), ratio_evap = (0.01, 0.99, 0.01), ratio_inf = (0.01, 0.99, 0.01), 
          t_sup = (0.8,2,0.1), t_sub = (2,10,0.1))

def modelo_hidrologico(sm_max=10, ratio_evap=0.5, ratio_inf=0.5, t_sup=1, t_sub=5): # valores iniciales de los parametros
    """
    Este modelo hidrologico es una adaptacion del modelo HyMOD. Tiene 5 parametros:

    sm_max: Maxima capacidad de retencion del del suelo (mm) [10-90]
    ratio_evap:  Ratio de evapotranspiracion () [0-1]
    ratio_inf:  Ratio de infiltracion () [0-1]
    t_sup:  Tiempo concentracion del flujo superficial [0.8 - 2]
    t_sub:  Tiempo concentracion del flujo subterraneo [2 - 10]
    """
    
    #######################################################################
    # Inicializar variables
    #######################################################################
    inf   = np.zeros((T,1))   # Lluvia infiltrada [mm/t]
    sup   = np.zeros((T,1))   # Lluvia acumulada en superficie [mm/t]
    et    = np.zeros((T,1))   # Tasa de evapotranspiration [mm/t]
    sm    = np.zeros((T+1,1)) # Contendio de humdedad en el suelo [mm] (suponemos que el suelo esta seco inicialmente)
    sL    = np.zeros((T+1,1)) # Slow reservoir moisture [mm]
    sF    = np.zeros((T+1,1)) # Fast reservoir moisture [mm]
    Q_sub = np.zeros((T,1))   # Flujo subterraneo [mm/t]
    Q_sup = np.zeros((T,1))   # Flujo superficial [mm/t]
    Q_tot = np.zeros((T,1))   # Flujo total [mm/t]

    #######################################################################
    # Simulacion
    #######################################################################
    for t in range(T):

        ##### Humedad del suelo #####          
        sm_temp = max(min(sm[t] + prec[t], sm_max), 0)

        ##### Lluvia infiltrada #####
        inf[t] = max(sm[t] + prec[t] - sm_max, 0) * ratio_inf
        
        ##### Lluvia acumulada en superficie #####
        sup[t] = max(sm[t] + prec[t] - sm_max, 0) * (1- ratio_inf)

        ##### Evapotranspiracion #####
        W = min(np.abs(sm[t]/sm_max)*ratio_evap, 1) # Factor de correccion de la evapotranspiracion
        et[t] = W * etp[t] # Calculo de la evapotranspiracion

        ##### Humedad del suelo (t+1) #####
        sm[t+1] = max(min(sm_temp - et[t], sm_max), 0)

        ##### Flujo subterraneo ######
        Q_sub[t] = 1/t_sub * sL[t]
        sL[t+1] = sL[t] + inf[t] - Q_sub[t]

        ##### Flujo superficial #####
        Q_sup[t] = 1/t_sup * sF[t]
        sF[t+1] = sF[t] +  sup[t] - Q_sup[t]

        ##### Flujo total #####
        Q_tot[t] = Q_sup[t] + Q_sub[t]

    #######################################################################
    # Visualizacion de los resultados
    #######################################################################
    # Dibujar la grafica
    plt.figure(figsize=(20,6))
    plt.plot(dates,Q_tot, label = 'hidrograma (flujo superficial + flujo subterraneo)', color = 'blue')
    plt.plot(dates,Q_sup, linestyle = '--', color = 'orange',   label = 'flujo superficial')
    plt.plot(dates,Q_sub, linestyle = ':',  color = 'green', label = 'flujo subterraneo')
    plt.ylim(0,25) # para fijar los limites del eje y, en este caso 0 y 25
    plt.legend()
    plt.show()

### Calibracion del modelo hidrologico
Un modelo hidrologico suele tener un gran número de parámetros. El usuario es quien decide el valor de estos parámetros para una aplicación particular en función de la información y datos que tengamos sobre los parámetros. Normalmente no vamos a tener mediciones directas de los valores de dichos parametros pero si es probable que tengamos datos climaticos historicos y del caudal del rio (hidrograma historico del rio). Con estos datos podemos inferir los valores de los parámetros encontrando los valores que hacen que el resultado del modelo se ajuste mejor al hidrograma historico del rio, esto se denomina **calibración** del modelo.

La forma más sencilla de hacer esto es cambiando los valores de los parámetros de uno en uno y observando cómo esto cambia el resultado del modelo y como de bien se ajusta al hidrograma historico. Esto es similar a lo que hicimos en el Notebook [Ejemplo - Onda de Sonido](Ejemplo%20-%20Onda%20de%20Sonido.ipynb), solo que ahora no simulamos una onda de sonido sino el caudal de un rio y ahora no tenemos 3 parametros (amplitud, fase, frecuencia) sino 5 parametros.

Intenta ahora, cambiando los valores de los 5 parametros, calibrar el modelo, es decir ajustar el hidrograma que simula el modelo al hidrograma historico.

In [ ]:
@interact(sm_max = (10, 90, 1), ratio_evap = (0.01, 0.99, 0.01), ratio_inf = (0.01, 0.99, 0.01), 
          t_sup = (0.8,2,0.1), t_sub = (2,10,0.1))

def modelo_hidrologico(sm_max=10, ratio_evap=0.5, ratio_inf=0.5, t_sup=1, t_sub=5): # valores iniciales de los parametros
    """
    Este modelo hidrologico es una adaptacion del modelo HyMOD. Tiene 5 parametros:

    sm_max: Maxima capacidad de retencion del del suelo (mm) [10-90]
    ratio_evap:  Ratio de evapotranspiracion () [0-1]
    ratio_inf:  Ratio de infiltracion () [0-1]
    t_sup:  Tiempo concentracion del flujo superficial [0.8 - 2]
    t_sub:  Tiempo concentracion del flujo subterraneo [2 - 10]
    """
    
    #######################################################################
    # Inicializar variables
    #######################################################################
    inf   = np.zeros((T,1))   # Lluvia infiltrada [mm/t]
    sup   = np.zeros((T,1))   # Lluvia acumulada en superficie [mm/t]
    et    = np.zeros((T,1))   # Tasa de evapotranspiration [mm/t]
    sm    = np.zeros((T+1,1)) # Contendio de humdedad en el suelo [mm] (suponemos que el suelo esta seco inicialmente)
    sL    = np.zeros((T+1,1)) # Slow reservoir moisture [mm]
    sF    = np.zeros((T+1,1)) # Fast reservoir moisture [mm]
    Q_sub = np.zeros((T,1))   # Flujo subterraneo [mm/t]
    Q_sup = np.zeros((T,1))   # Flujo superficial [mm/t]
    Q_tot = np.zeros((T,1))   # Flujo total [mm/t]

    #######################################################################
    # Simulacion
    #######################################################################
    for t in range(T):

        ##### Humedad del suelo #####          
        sm_temp = max(min(sm[t] + prec[t], sm_max), 0)

        ##### Lluvia infiltrada #####
        inf[t] = max(sm[t] + prec[t] - sm_max, 0) * ratio_inf
        
        ##### Lluvia acumulada en superficie #####
        sup[t] = max(sm[t] + prec[t] - sm_max, 0) * (1- ratio_inf)

        ##### Evapotranspiracion #####
        W = min(np.abs(sm[t]/sm_max)*ratio_evap, 1) # Factor de correccion de la evapotranspiracion
        et[t] = W * etp[t] # Calculo de la evapotranspiracion

        ##### Humedad del suelo (t+1) #####
        sm[t+1] = max(min(sm_temp - et[t], sm_max), 0)

        ##### Flujo subterraneo ######
        Q_sub[t] = 1/t_sub * sL[t]
        sL[t+1] = sL[t] + inf[t] - Q_sub[t]

        ##### Flujo superficial #####
        Q_sup[t] = 1/t_sup * sF[t]
        sF[t+1] = sF[t] +  sup[t] - Q_sup[t]

        ##### Flujo total #####
        Q_tot[t] = Q_sup[t] + Q_sub[t]

    #######################################################################
    # Visualizacion de los resultados
    #######################################################################
    # Cargar los datos historicos del caudal del rio (hidrograma historico)
    from modulos.hidrograma_historico import hidrograma_historico
    Q_tot_hist = hidrograma_historico(T,prec,etp)
    # Dibujar la grafica
    plt.figure(figsize=(20,7))
    plt.plot(dates,Q_tot_hist, label = 'hidrograma historico', color = 'black')
    plt.plot(dates,Q_tot, label = 'hidrograma simulado', color = 'blue')
    plt.ylim(0,25) # para fijar los limites del eje y, en este caso 0 y 25
    plt.legend()
    plt.show()

Entender el concepto de calibracion de un modelo y para los que ya sabeis que es la calibracion, como ayuda a comprender el modelo en sí. Que significa cada parametro, que parametros influencian mas el resultado y comprobar que el model se comporta de una manera logica. Y con elementos interactivos como los widgets facilitamos esta tarea y por tanto la comprension del modelo (por ejemplo hidrologico) o de una ecuacion (por ejemplo, la de una onda de sonido).